In [1]:
import pygad 
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal, skew
from numba import int32, float64   # import the types
from portfolio_simulator import PortfoliosSimulator

In [42]:
def expected_returns(period):
    return pd.Series({
		'ACWI': 0.07/period,
		'^TBSP': 0.035/period,
		'XAUPLN': 0.05/period,
		'MWIG40': 0.065/period,
		'IHYU.L': 0.038/period,
		'PLOPLN3M': 0.03/period,
		'edo': 0.032/period
})

""" cov = np.array([[0.001265,	0.000042],
            	[0.000042,	0.000164]]) """
#acwi edo
cov = np.array([[0.030276,	0.0],
            	[0.0,	0.00021]])
cov_y = cov 

assets = ['ACWI', 'edo']
means = expected_returns(1)[assets]
number_of_scenarios = 10000
T = 50
sim_returns = []

for s in range (number_of_scenarios):
    sim_returns.append(multivariate_normal.rvs(means,cov_y, T))

sim_returns = np.array(sim_returns)
sim_returns.shape

(10000, 50, 2)

In [57]:
class Simulation():
    def __init__(self, initial_value, inflows, returns) -> None:
        self.initial_value = initial_value
        self.returns = returns         
        self.inflows = inflows
        
    def step(self, current_value,inflow,weights,returns):
        current_assets_value = np.expand_dims(current_value+inflow,axis=0).T * weights
        next_assets_value =  np.round(current_assets_value * (1+returns),2)
        return next_assets_value.sum(1)
         
    def run(self, strategy):
        self.capital = self.initial_value
        T = len(strategy)
        for t in range(T):
            self.capital = self.step(self.capital,self.inflows[t],strategy[t],self.returns[:,t])

In [4]:
portfolios = [[0.0,1.0],
             [0.1,0.9],
             [0.2,0.8],
             [0.3,0.7],
             [0.4,0.6],
             [0.5,0.5],
             [0.6,0.4],
             [0.7,0.3],
             [0.8,0.2],
             [0.9,0.1],
             [1.0,0.0]] 

In [45]:
from typing import Any

class MaxPercentile():

    def __init__(self, percentile) -> None:
        self.percentile = percentile

    def __call__(self, values: np.array) -> float:
        return np.round(np.percentile(values,self.percentile))

class Fitness:
    def __init__(self,simulator: Simulation,portfolios,objective) -> None:
        self.simulator = simulator
        self.portfolios = portfolios
        self.objective = objective

    def __call__(self, solution: np.array) -> Any:
        strategy = np.take(self.portfolios,solution, axis=0)
        self.simulator.run(strategy)
        result = self.objective(self.simulator.capital)
        return result

In [29]:
objective = MaxPercentile(50)
inflows = np.cumproduct(np.full(T,1.02))*0
simulator = Simulation(1,inflows,sim_returns)
fitness = Fitness(simulator,portfolios,objective)

solution = np.full(10,6)

fitness(solution)

2.0

In [53]:
def optimize_glide_path(T, capital, inflows,risk, portfolios, returns):
    l = np.arange(T,0,-1)
    
    glide_paths = np.full(T+1,len(portfolios))
    perc = np.zeros((l.shape[0],len(portfolios)))

    objective = MaxPercentile(risk)
    simulator = Simulation(capital,inflows,returns)
    fitness = Fitness(simulator,portfolios,objective)

    for t in l:
        
        for portfolio in range(len(portfolios)):           
            
            solution = np.full(t,portfolio)
            
            #perc.append(np.percentile(simulator.capital,risk))
            perc[t-1,portfolio] = fitness(solution)
        glide_paths[t-1] = np.argmax(perc[t-1])

    #glide = list(glide_paths.values())
    return np.flip(glide_paths)[1:]

In [60]:
T = 20
capital = 10
risk = 30
inflows = np.cumproduct(np.full(T,1.02))*5

optimize_glide_path(T,capital,inflows,risk,portfolios,sim_returns)

array([6, 6, 5, 6, 5, 5, 4, 2, 3, 2, 3, 2, 1, 0, 1, 1, 0, 0, 0, 0])

array([8, 7, 6, 5, 8, 6, 7, 6, 5, 6, 7, 6, 6, 6, 4, 6, 6, 5, 5, 3, 3, 3,
       2, 2, 2, 2, 1, 1, 1, 1])